#### Importing necessary libraries

In [40]:
import numpy as np
import pandas as pd
import plotly.express as px

In [41]:
df1 = pd.read_excel('model/data-welding.xlsx', sheet_name='Sheet1')
df2 = pd.read_excel('model/data-welding.xlsx', sheet_name='Sheet2')

In [42]:
df1.head()

,Weld No.,V,A,S,N,Bead height (H),Bead width (W),Bead Penetration (P)
0,1,-1,-1,-1,-1,2.38,10.55,3.94
1,2,1,-1,-1,-1,2.54,12.43,5.08
2,3,-1,1,-1,-1,3.07,9.76,5.12
3,4,1,1,-1,-1,2.79,13.14,6.40
4,5,-1,-1,1,-1,2.03,10.27,3.05


In [43]:
df2.head()

,Sl.No,PARAMETERS,Units,Notation,value(-2),value(-1),value(0),value(1),value(2)
0,1,Current,Amperes,A,250,325.0,400,475.0,550
1,2,Arc Voltage,Volts,V,22,22.5,29,32.5,36
2,3,Travel speed,mm/sec,S,6,9.0,12,15.0,18
3,4,Nozzle to plate distance,Mm,N,20,23.5,27,30.5,34


In [44]:
df2 = df2.set_index('Notation')
df2

,Sl.No,PARAMETERS,Units,value(-2),value(-1),value(0),value(1),value(2)
Notation,,,,,,,,
A,1,Current,Amperes,250,325.0,400,475.0,550
V,2,Arc Voltage,Volts,22,22.5,29,32.5,36
S,3,Travel speed,mm/sec,6,9.0,12,15.0,18
N,4,Nozzle to plate distance,Mm,20,23.5,27,30.5,34


In [45]:
data = df1.copy()

In [46]:
for i in range(len(df1)):
    for col in ['V', 'A', 'S', 'N']:
        data.loc[i, col] = df2[f'value({str(int(df1[col][i]))})'][col]

C:\Users\aritr\AppData\Local\Temp\ipykernel_8924\2874447586.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '22.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

C:\Users\aritr\AppData\Local\Temp\ipykernel_8924\2874447586.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '23.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [47]:
data.head()

,Weld No.,V,A,S,N,Bead height (H),Bead width (W),Bead Penetration (P)
0,1,22.5,325,9,23.5,2.38,10.55,3.94
1,2,32.5,325,9,23.5,2.54,12.43,5.08
2,3,22.5,475,9,23.5,3.07,9.76,5.12
3,4,32.5,475,9,23.5,2.79,13.14,6.40
4,5,22.5,325,15,23.5,2.03,10.27,3.05


In [48]:
df1.head()

,Weld No.,V,A,S,N,Bead height (H),Bead width (W),Bead Penetration (P)
0,1,-1,-1,-1,-1,2.38,10.55,3.94
1,2,1,-1,-1,-1,2.54,12.43,5.08
2,3,-1,1,-1,-1,3.07,9.76,5.12
3,4,1,1,-1,-1,2.79,13.14,6.40
4,5,-1,-1,1,-1,2.03,10.27,3.05


In [49]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [50]:
X = data.iloc[:, 1:5].values
y = data.iloc[:, -3:].values

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=316)

In [52]:
regressor = XGBRegressor()
regressor.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [53]:
y_pred = regressor.predict(X_test)

In [54]:
y_pred

array([[ 3.0605316,  8.962853 ,  5.332557 ],
       [ 1.6624138, 10.807027 ,  2.8025804],
       [ 2.0342035, 10.7027445,  4.0859714],
       [ 2.978788 , 12.786759 ,  5.806867 ],
       [ 1.8098342,  9.049013 ,  3.023615 ],
       [ 2.0185375, 11.0030365,  3.9628763],
       [ 2.1430464,  9.755803 ,  3.8631697]], dtype=float32)

In [55]:
y_test

array([[ 3.23,  9.48,  7.31],
       [ 2.08,  9.74,  2.08],
       [ 1.96, 10.4 ,  4.34],
       [ 2.79, 13.14,  6.4 ],
       [ 2.09,  8.08,  2.6 ],
       [ 1.93, 11.4 ,  3.94],
       [ 1.98,  9.26,  3.59]])

In [56]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.7848029133381121

In [57]:
test_data = pd.read_excel('model/data-welding.xlsx', sheet_name='Sheet3')
test_data

,Weld no.,V,A,S,N,H,W,P
0,5,25.5,325,15,23.5,2.03,10.27,3.05
1,8,32.5,475,15,23.5,1.99,9.42,5.37
2,16,32.5,475,15,30.5,3.05,10.58,4.19
3,23,29.0,400,12,20.0,1.96,10.40,4.34
4,31,29.0,400,12,27.0,1.94,10.72,3.68


In [58]:
pred = regressor.predict(test_data.iloc[:, 1:5])
pred

array([[ 2.0312629, 10.270511 ,  3.0516596],
       [ 1.9897397,  9.421179 ,  5.369852 ],
       [ 3.0491655, 10.578496 ,  4.192153 ],
       [ 2.0342035, 10.7027445,  4.0859714],
       [ 2.0185375, 11.0030365,  3.9628763]], dtype=float32)

In [59]:
actual = test_data.iloc[:, -3:]
actual

,H,W,P
0,2.03,10.27,3.05
1,1.99,9.42,5.37
2,3.05,10.58,4.19
3,1.96,10.40,4.34
4,1.94,10.72,3.68


In [60]:
percentage_error = (pred - actual) / actual * 100
percentage_error

,H,W,P
0,0.062211,0.004972,0.054413
1,-0.013083,0.012514,-0.002755
2,-0.027361,-0.014216,0.051384
3,3.785894,2.911005,-5.853195
4,4.048326,2.640266,7.686857


In [61]:
import pickle

filename = 'finalized_model.sav'
# pickle.dump(regressor, open(filename, 'wb'))

regressor.save_model(f'model/{filename}')
regressor.load_model(f'model/{filename}')
regressor.predict(X_test)

c:\Users\aritr\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning:

[00:09:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.



array([[ 3.0605316,  8.962853 ,  5.332557 ],
       [ 1.6624138, 10.807027 ,  2.8025804],
       [ 2.0342035, 10.7027445,  4.0859714],
       [ 2.978788 , 12.786759 ,  5.806867 ],
       [ 1.8098342,  9.049013 ,  3.023615 ],
       [ 2.0185375, 11.0030365,  3.9628763],
       [ 2.1430464,  9.755803 ,  3.8631697]], dtype=float32)

## Graphs

In [62]:
import plotly.express as px
fig = px.scatter(data, x="V", y=["Bead height (H)", "Bead width (W)", "Bead Penetration (P)"], trendline="lowess") 
# LOESS (locally estimated scatterplot smoothing) and LOWESS (locally weighted scatterplot smoothing)
fig.show()

In [63]:
fig = px.scatter(data, x="A", y=["Bead height (H)", "Bead width (W)", "Bead Penetration (P)"], trendline="lowess")
fig.show()

In [64]:
fig = px.scatter(data, x="S", y=["Bead height (H)", "Bead width (W)", "Bead Penetration (P)"], trendline="lowess")
fig.show()

In [65]:
fig = px.scatter(data, x="N", y=["Bead height (H)", "Bead width (W)", "Bead Penetration (P)"], trendline="lowess")
fig.show()

In [66]:
@np.vectorize
def f1(A, N):
    return regressor.predict([[0, A, 0, N]])[0][0]

@np.vectorize
def f2(S, N):
    return regressor.predict([[0, 0, S, N]])[0][0]

@np.vectorize
def f3(V, A):
    return regressor.predict([[V, A, 0, 0]])[0][1]

@np.vectorize
def f4(V, S):
    return regressor.predict([[V, 0, S, 0]])[0][1]

In [67]:
import plotly.graph_objects as go

In [68]:
x = data['A'].sort_values()
y = data['N'].sort_values()
xx, yy = np.meshgrid(x, y)
zz = f1(xx, yy)

fig = go.Figure(data =
    go.Surface(
        z=zz,
        x=x,
        y=y
    ))
fig.update_layout(scene = dict(
                    xaxis_title='A',
                    yaxis_title='N',
                    zaxis_title='H'))
fig.show()

In [69]:
x = data['S'].sort_values()
y = data['N'].sort_values()
xx, yy = np.meshgrid(x, y)
zz = f2(xx, yy)

fig = go.Figure(data =
    go.Surface(
        z=zz,
        x=x,
        y=y
    ))
fig.update_layout(scene = dict(
                    xaxis_title='S',
                    yaxis_title='N',
                    zaxis_title='H'))
fig.show()

In [70]:
x = data['V'].sort_values()
y = data['A'].sort_values()
xx, yy = np.meshgrid(x, y)
zz = f3(xx, yy)

fig = go.Figure(data =
    go.Surface(
        z=zz,
        x=x,
        y=y
    ))
fig.update_layout(scene = dict(
                    xaxis_title='V',
                    yaxis_title='A',
                    zaxis_title='W'))
fig.show()

In [71]:
x = data['V'].sort_values()
y = data['S'].sort_values()
xx, yy = np.meshgrid(x, y)
zz = f4(xx, yy)

fig = go.Figure(data =
    go.Surface(
        z=zz,
        x=x,
        y=y
    ))
fig.update_layout(scene = dict(
                    xaxis_title='V',
                    yaxis_title='S',
                    zaxis_title='W'))
fig.show()